In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from circuit import Circuit
import numpy as np
import dgl.function as fn
import random
import networkx as nx
from utils import *
from dgl.data.utils import save_graphs, load_graphs
import time

Using backend: pytorch


In [2]:
import os.path
preprocess_begin = time.time()

design = "tate_GNN"
path = design+"/heterograph.bin"

cir = Circuit(design)
root = "/autofs/home/sh528/M3Ddesigns/"+design+"/"
cir.parseHierVerilog(root+"die0.v")
cir.parseHierVerilog(root+"die1.v")
cir.parseTop(root+"top.v")
# cir.parseVerilog(design+"/"+design+".v")
# cir.parsePartition(design+"/die0.rpt")
stil = design+"/TDF.stil"

# if False:
if os.path.isfile(path):
    n_patterns = cir.parseSTIL(stil, -2)
    dic, topEdge = backprop(cir,False)
    hg = load_graphs(path)[0][0]
else:
    n_patterns = cir.parseSTIL(stil)
    edge = CreateGraphByFaultSite(cir)
    dic, topEdge = backprop(cir)
    

    hg = dgl.heterograph({ ('topNode', 'topEdge', 'faultSite'): topEdge, ('faultSite', 'net', 'faultSite'): edge })
    feats = torch.tensor([cir.Node[n].net.feats for n in cir.Node])
    hg.nodes['faultSite'].data['feats'] = feats
    hg.nodes['faultSite'].data['in_degree'] = hg.in_degrees(etype='net').view(-1,1).float()
    hg.nodes['faultSite'].data['out_degree'] = hg.out_degrees(etype='net').view(-1,1).float()
    hg.nodes['faultSite'].data['top_degree'] = hg.in_degrees(etype='topEdge').view(-1,1).float()
    hg.nodes['faultSite'].data['level'] = getLevel(cir)
    hg.nodes['faultSite'].data['loc'] = getLocation(cir, hg.num_nodes('faultSite'))
    hg.nodes['faultSite'].data['more'] = addfeatures(cir, hg.num_nodes('faultSite'))
    save_graphs(path, hg)
#     fout = open(design+"/topNode.dic", "w")
#     for n in dic:
#         fout.write("{},{}\n".format(n,dic[n]))
    
print(hg)

Start parsing verilog netlist
nodeID: 492495
End parsing verilog netlist
CPU time: 6.05s

Start parsing verilog netlist
nodeID: 897182
End parsing verilog netlist
CPU time: 4.72s

Start parsing top verilog netlist
End parsing verilog netlist
CPU time: 3.32s

Start parsing STIL patterns
Pass Pattern 0
Final Pat
End parsing STIL patterns
CPU time: 2.53s

Graph(num_nodes={'faultSite': 1184940, 'topNode': 31409},
      num_edges={('faultSite', 'net', 'faultSite'): 1354733, ('topNode', 'topEdge', 'faultSite'): 3447617},
      metagraph=[('faultSite', 'faultSite', 'net'), ('topNode', 'faultSite', 'topEdge')])


In [3]:
from dgl.nn.pytorch import GraphConv
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, 16)
        self.linear1 = nn.Linear(16+3, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        
        ratio = torch.sum(g.ndata['infeats'][:,3:6], dim=0)/g.num_nodes()

        g.ndata['h_final'] = h
        h_final = dgl.readout_nodes(g, 'h_final', op='max')
        h_final = F.softmax(h_final, dim=0)
        h_final = torch.cat([h_final, ratio.unsqueeze(0)], dim=1)

        return self.linear1(h_final)

In [4]:
PATH = design + "/saved_model"
model = GCN(11,128,3)
model.load_state_dict(torch.load(PATH))
model.eval()

GCN(
  (conv1): GraphConv(in=11, out=128, normalization=both, activation=None)
  (conv2): GraphConv(in=128, out=16, normalization=both, activation=None)
  (linear1): Linear(in_features=19, out_features=3, bias=True)
)

In [21]:
def getSample(cir, design, dic, g, num_patterns, logname, start_pat=0, end_pat=-1):
    
    
    words = logname.split("/")[-1].rsplit("_",2)
    gname = words[0]
    pname = words[1]

#     if words[1].split("/")[-1] == "nextstate":
#         pname = "D"
#     elif words[1].split("/")[-1] == "IQ":
#         pname = "Q"

    if gname.startswith("MIV") and pname == "A":
        pname = "Z"

    dstID = cir.Node[gname+"_"+pname].ID
    label = -1
    if g.nodes['faultSite'].data['loc'][dstID][0] == 1:
        label = 0
    elif g.nodes['faultSite'].data['loc'][dstID][1] == 1:
        label = 1
    else:
        label = 2

    if not os.path.isfile(logname):
        print("Logfile does not exist!")
        return 0,0

    f2 = open(logname, "r")
    l2 = f2.readlines()[1:]
    f2.close()


    num_pat = end_pat-start_pat
    success = True
    subnodes = []
    for fault in l2:
        w2 = fault.split()
        if len(w2) != 5:
            success = False
            break
        pat = int(w2[0])-1

        if pat < start_pat:
            continue

        if pat >= end_pat:
            break

        chname = w2[1]
        loc = int(w2[2])


        chain = cir.scanchains[cir.sopin.index(chname)]
        gname = chain[::-1][loc].name
        srcID = dic[gname]

        tmpnodes =  g.successors(srcID, etype=('topNode', 'topEdge', 'faultSite')).numpy()

        if len(subnodes):
            tmpnodes = np.intersect1d(subnodes, tmpnodes)

        subnodes = np.array([idx for idx in tmpnodes if g.nodes['faultSite'].data['feats'][idx][pat-start_pat] == 1.0])

        if not len(subnodes):
            break


    if not success:
        print("Not success!")
        return 0,0
    if not len(subnodes):
        print("No candidates!")
        return 0,0

    with g.local_scope():
        sg = g.subgraph({'faultSite': subnodes})

        assert(dstID in sg.ndata[dgl.NID]['faultSite'])

        infeats = torch.cat([sg.nodes['faultSite'].data['in_degree'], sg.nodes['faultSite'].data['out_degree'], sg.nodes['faultSite'].data['top_degree']], dim=1)
        infeats = torch.cat([infeats, sg.nodes['faultSite'].data['loc']], dim=1)
        infeats = torch.cat([infeats, sg.nodes['faultSite'].data['level']], dim=1)
        infeats = torch.cat([infeats, sg.nodes['faultSite'].data['more']], dim=1)
        infeats = torch.cat([infeats, sg.in_degrees(etype='net').view(-1,1).float()], dim=1)
        infeats = torch.cat([infeats, sg.out_degrees(etype='net').view(-1,1).float()], dim=1)

        sg = dgl.to_homogeneous(sg)
        sg = dgl.add_reverse_edges(sg)
        sg.ndata['infeats'] = infeats

    sg = dgl.add_self_loop(sg)
            
    return sg, label

In [26]:
inference_st = time.time()

log = "U237924_A_stf.log"
logpath = design+"/Logs_w_MIV/"+log
g, l = getSample(cir, design, dic, hg, n_patterns, logpath, 0, n_patterns)

with torch.no_grad():
    infeats = g.ndata['infeats']
    pred = model(g, infeats)
    print("Acutual Location: {}, Predictied Location: {}".format(l, pred.argmax(1)))

inference_end = time.time()

print("Total CPU time for inferencing: {:.3f}s".format(inference_end-inference_st))

Acutual Location: 1, Predictied Location: tensor([2])
Total CPU time for inferencing: 0.053s
